<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Parsing-and-Basics" data-toc-modified-id="Parsing-and-Basics-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parsing and Basics</a></span></li><li><span><a href="#Compare-to-other-genome-sequences" data-toc-modified-id="Compare-to-other-genome-sequences-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Compare to other genome sequences</a></span></li><li><span><a href="#Extract-annotations-on-the-matching-genome-sequence" data-toc-modified-id="Extract-annotations-on-the-matching-genome-sequence-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Extract annotations on the matching genome sequence</a></span></li><li><span><a href="#Protein-level-analyses" data-toc-modified-id="Protein-level-analyses-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Protein level analyses</a></span><ul class="toc-item"><li><span><a href="#What's-next?" data-toc-modified-id="What's-next?-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>What's next?</a></span></li></ul></li></ul></div>

There is an unusual outbreak of diseases in your city. What causes it is unknown, but a sequence has been found in all the patients. Your are tasked with investigating the causes. Luckily, you have just completed a course in Python and are eager to try out what you've learned.

Fill out the code fields as instructed by the comments. If you need help finding the function's parameter use Jupyter's "function?" command or ask Google. Refer to BioPython's readme or google what you need if you're stuck. If all else fails: Ask first your groupmates, then your instructor for help. ;)

# Setup

You go ahead and import relevant packages from Biopython:

In [1]:
import os
import sys

import Bio
from Bio import SeqIO, SearchIO, Entrez, AlignIO
from Bio.Seq import Seq
from Bio.SeqUtils import GC
from Bio.Blast import NCBIWWW
from Bio.Data import CodonTable

You then find the fasta file with the unknown sequence. Check out the file size first. If it's small enough, check it out in your editor before you load it into Python.

In [2]:
input_file = "unknown-sequence.fa"

To get an overview, we display the ID of every sequence in the file:

In [3]:
for record in SeqIO.parse(input_file, "fasta"):
    print(record.id)

Unknown_sequence


While this is not terribly helpful, we now know there is only one sequence.

# Parsing and Basics

We start out by parsing the file:

In [4]:
# use SeqIO.read to parse the file
# file type is "fasta"
record = SeqIO.read(input_file, 'fasta')

In [5]:
# check what the type of the returned object is
type(record)

Bio.SeqRecord.SeqRecord

This yields a Biopython SeqRecord object. See here for more details: https://biopython.org/wiki/SeqRecord

In [6]:
print(record)

ID: Unknown_sequence
Name: Unknown_sequence
Description: Unknown_sequence
Number of features: 0
Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...GTT', SingleLetterAlphabet())


In [7]:
# find out the length of the sequence, hint: Python's normal length function works on SeqRecord objects
len(record)

2520

In [8]:
record.seq

Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...GTT', SingleLetterAlphabet())

The sequence length is ~30Kb, if this sequence represents an individual organism then it is very small. Far too small for a typical eukaryote and in fact too short many microbes too (e.g. bacterial genomes are typically Mb). You have a dim feeling what this sequence may be...

In [9]:
# find out the GC content of the sequence. Hint: You've imported a function called GC.
# Can it be called on the SeqRecord? Or maybe one of its attributes?
GC(record.seq)

42.142857142857146

So the sequence is somewhat AT-rich, but within a 'normal' range.

# Compare to other genome sequences

You decide to run the sequence through Blast to see if you can find any close relatives. This may take ~10 minutes since we are doing an online search against many sequences. But for now it will do and it saves you from downloading the databases. (BioPython can run Blast for you locally as well.)

In [10]:
#%%time
# query blast using the function NCBIWWWW.qblast
# think about and discuss which blast and which database to use first
# make doubly sure to use the correct parameters as mistakes cost time
#result_handle = NCBIWWW.qblast('blastn', 'nt', record.seq)

That took long enough... You're eager to check out the results. Luckily, BioPython can also handle the parsing for you:

In [11]:
# now we parse the blast results using BioPython's SearchIO.read
# the standard format for the result is blast-xml
blast_qresult = SearchIO.read('blast_search.xml', 'blast-xml')
print(blast_qresult)

Program: blastn (2.14.0+)
  Query: Unknown_sequence (2520)
 Target: nt
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gi|2288631864|gb|OP278726.1|  Severe acute respiratory ...
            1      1  gi|2288045599|gb|OP268178.1|  Severe acute respiratory ...
            2      1  gi|2274240152|gb|OP022337.1|  Severe acute respiratory ...
            3      1  gi|2274240139|gb|OP022336.1|  Severe acute respiratory ...
            4      1  gi|2274240126|gb|OP022335.1|  Severe acute respiratory ...
            5      1  gi|2274240113|gb|OP022334.1|  Severe acute respiratory ...
            6      1  gi|2274240099|gb|OP022333.1|  Severe acute respiratory ...
            7      1  gi|2274240086|gb|OP022332.1|  Severe acute respiratory ...
            8      1  gi|2274240072|gb|OP022331.1|  Severe acute respiratory ...

Those descriptions are truncated, let's view them in full, for just the first 5 records

In [21]:
# blast_qresult is an iterable. iterate over the first 5 of its elements and print element.description
for hit in blast_qresult[:5]:
    print(hit.description)

Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/PAK/WA-UW-1729/2020, complete genome
Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/MEX/Wuhan-Hu-1/2022, complete genome
Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/NMINBRE-ILLUMINA-WGS-250930/2021, complete genome
Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/NMINBRE-ILLUMINA-WGS-236204/2020, complete genome
Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/NMINBRE-ILLUMINA-WGS-215866/2020, complete genome


Without doing any quantitative analyses, it already seems very likely we have a coronavirus genome here, specifically SARS2-CoV-2 that was the cause of the COVID19 pandemic!

Let's have a look at the first result in a bit more detail to check some of the alignment metrics

In [13]:
first_hit = blast_qresult[0]

In [14]:
first_hit.description

'Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/PAK/WA-UW-1729/2020, complete genome'

In [15]:
# What kind of object is first_hit? Can you find any information on it?

In [16]:
# extract the first hsp from the hit

# Find the e-Value associated with that HSP

In [17]:
# print the alignment. hint: it's an attribute of first_hsp

Let's save the alignment to have a better look at it and for future reference:

In [18]:
# write the alignment to file using AlignIO.write, use the "clustal" format

# Extract annotations on the matching genome sequence

Let's extract a bit more structured meta-data on the top matching sequence homologous sequence using NCBI Entrez via Biopython to extract a GenBank file

In [19]:
print(first_hit.id)

gi|2288631864|gb|OP278726.1|


In [20]:
NCBI_id = # they NCBI ID is the fourht element in the string. Can you extract just that?
NCBI_id

SyntaxError: invalid syntax (<ipython-input-20-2ba7c9af3382>, line 1)

In [ ]:
Entrez.email = "A.N.Other@example.com"  # tell NCBI who you are; promise there's no spam mails

In [ ]:
handle = # use the Entrez.efetch function to extract the information for the Gene ID
# for the params, use retmode="text" and rettype="gb"
handle

In [ ]:
genbank_record = # read in the handle using SeqIO.read, the format is "genbank"
genbank_record

There's a lot of information in the genbank record if you know where to find it...

First you decide to double-check whether the virus has single- or double-stranded DNA or RNA:

In [ ]:
# Check in the genbank_record whether the Sequence is ss or ds, RNA or DNA

That's odd. Your original sample was DNA. Any idea why?

You decide to investigate the Virus's full taxonomy:

In [ ]:
# check in the annotations again

Lastly, you're curious who you have to thank for all this data:

In [ ]:
# Check which publications all this data came from

That should be more than enough information for a bit.

Note that from this id, we could also find the [record here](https://www.ncbi.nlm.nih.gov/nuccore/NC_045512.2/) on the NCBI website.

# Protein level analyses

You're also interested in the gene/protein sequences, not just the entire genome. Luckily for you, BioPython makes it easy to retrieve the protein coding sequences (CDSs) from the Genbank record:

In [ ]:
#check how many and which "features" the genbank record contains, hint: p

In [ ]:
# each feature has a type attribute. check what they are for each feature

You extract the coding sequences to have a closer look at them:

In [ ]:
CDSs = # extract all features that are coding sequences (CDS)
# how many CDSs have been found?

You check out the first protein in detail and extract the underlying sequence:

In [ ]:
# print the "gene" qualifier for every CDS

In [ ]:
# how many peptides does the sequence translate to? see qualifier "translation"
protein_seq = # select the first protein translation
protein_seq = # make the string into a BioPython Sequence object
protein_seq

Refer to https://biopython.org/wiki/Seq for details of the Seq object.

The sequence should begin with a start codon, right?

First, you look up possible starting amino acids.

In case you needed a refresher on the codon table:

In [ ]:
print(CodonTable.unambiguous_dna_by_id[1])

In [ ]:
# Find if the sequence starts with a fitting amino acid

However, we can't perform an exact "reverse translation" of course, since several amino acids are produced by the same codon. Note that if instead we started with the nucleotide sequence, then we could use Biopython's `.transcribe()` and `.translate()` functions to convert sequences from DNA to RNA and DNA to protein respectively.

In any case, whe can get the sequence length in amino acids:

In [ ]:
# get the sequence length in amino acids

That's quite a long protein for a virus. You decide to check the annotation:

In [ ]:
# check the "product" qualifier in the CDS attributes

So it looks like this is a polyprotein, which explains why it is a relatively long protein. Polyproteins are a typical feature of some viral genomes where smaller proteins are joined together, providing a particular organization of the viral proteome.

## What's next?

Logical next steps at the genome level might include building a multiple sequence alignment from many cornavirus genomes (checkout the Biopython wrapers/parsers for `Clustal` and `Mafft` and `Bio.Align`/`Bio.parirwise2` plus `Bio.AlignIO`), building a phylogeny with an external tool like [iq-tree](http://www.iqtree.org/) and then viewing the tree with `Bio.Phylo`, the [ete3 toolkit](http://etetoolkit.org/), or [Jalview](https://www.jalview.org/).

Other protein level analyses could involve including building protein trees, annotating the proteins and vizulisation (e.g. `Bio.Graphics`), doing evolutionary rate analyses (e.g. `Bio.Phylo.PAML `), looking at protein structure, clustering and much more.

These kind of analyses can provide useful biological and epidemiological information, very important for this coronavirus in an outbreak situation. For example, allowing tracking of how the outbreak spreads and indicating appropriate infection control measures, although caution in the interpretation of results is always required. See [Nextstrain](https://nextstrain.org/ncov) for an excellent resource of this kind.

Note, there is tons of other functionality in Biopython, this is just a very small fraction of the modules, see [Peter Cock's Biopython workshop](https://github.com/peterjc/biopython_workshop) and the extensive [official tutorial documentation](http://biopython.org/DIST/docs/tutorial/Tutorial.html).